In [1]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from numpy.polynomial import polynomial as P

from ugradiolab.data import Record
from ugradiolab.analysis import Spectrum

HI_REST_HZ = 1420.405751768e6   # HI rest frequency
C          = 2.998e8            # speed of light, m/s

LAB2_1_DIR = Path('../../data/lab2_1')

lo_records = sorted(
    [Record.load(p) for p in LAB2_1_DIR.glob('*.npz') if 'CAL' not in p.stem],
    key=lambda r: r.center_freq,
)
lo_spectra = [Spectrum.from_record(r) for r in lo_records]

for r in lo_records:
    hi_offset = (HI_REST_HZ - r.center_freq) / 1e6
    print(f'  LO={r.center_freq/1e6:.0f} MHz   HI offset={hi_offset:+.3f} MHz')

/home/ikaros/projects/ay-121/.venv/lib/python3.12/site-packages/rtlsdr/__init__.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


  LO=1418 MHz   HI offset=+2.406 MHz
  LO=1419 MHz   HI offset=+1.406 MHz
  LO=1420 MHz   HI offset=+0.406 MHz
  LO=1421 MHz   HI offset=-0.594 MHz
  LO=1422 MHz   HI offset=-1.594 MHz
  LO=1423 MHz   HI offset=-2.594 MHz


In [2]:

# Build a common frequency grid spanning the intersection of all LO bands,
# sampled at the native channel spacing of the first spectrum.
# Each LO covers sample_rate = 2.56 MHz centred at its LO frequency.

f_axes = [spec.freqs for spec in lo_spectra]   # absolute Hz, DC-centred

f_common = np.linspace(
    max(f.min() for f in f_axes),
    min(f.max() for f in f_axes),
    lo_spectra[0].record.nsamples,
)
f_common_mhz = f_common / 1e6

print(f'Common frequency range: {f_common_mhz.min():.4f} to {f_common_mhz.max():.4f} MHz')
print(f'Span: {f_common_mhz.max() - f_common_mhz.min():.4f} MHz')
print(f'HI rest frequency: {HI_REST_HZ/1e6:.6f} MHz  ({"inside" if f_common.min() < HI_REST_HZ < f_common.max() else "outside"} common range)')


Common frequency range: 1419.2798 to 1421.7200 MHz
Span: 2.4402 MHz
HI rest frequency: 1420.405752 MHz  (inside common range)


In [3]:

BASELINE_DEG = 5   # polynomial degree for baseline fit

# ~150 km/s half-width in Hz: Δf = v/c * f_HI
HI_HALF_WIDTH_HZ = 150e3 / C * HI_REST_HZ   # ≈ 710 kHz


def subtract_baseline(f, psd, hi_half_width=HI_HALF_WIDTH_HZ):
    """Fit a polynomial to emission-free regions and subtract it.

    Parameters
    ----------
    f : array, frequency axis in Hz
    psd : array, power spectrum
    hi_half_width : float, half-width in Hz to mask around HI rest frequency
    """
    mask = np.abs(f - HI_REST_HZ) > hi_half_width   # True = emission-free

    # Normalise to [-1, 1] for numerical stability
    f_centre = f.mean()
    f_scale  = 0.5 * (f.max() - f.min())
    f_norm   = (f - f_centre) / f_scale

    coeffs   = P.polyfit(f_norm[mask], psd[mask], deg=BASELINE_DEG)
    baseline = P.polyval(f_norm, coeffs)
    return psd - baseline, baseline


# Interpolate each spectrum onto the common frequency grid, then subtract baseline
psd_on_common      = []
baseline_on_common = []

for spec, f_ax in zip(lo_spectra, f_axes):
    # Mask DC/LO bin before interpolating
    lo_bin = spec.bin_at(spec.record.center_freq)
    psd = spec.psd.copy()
    psd[lo_bin] = np.nan

    # Interpolate onto common grid
    finite = np.isfinite(psd)
    interp = interp1d(f_ax[finite], psd[finite],
                      bounds_error=False, fill_value=np.nan)
    psd_interp = interp(f_common)

    # Baseline subtraction
    finite2  = np.isfinite(psd_interp)
    residual = np.full_like(psd_interp, np.nan)
    bl       = np.full_like(psd_interp, np.nan)
    r, b = subtract_baseline(f_common[finite2], psd_interp[finite2])
    residual[finite2] = r
    bl[finite2]       = b

    psd_on_common.append(residual)
    baseline_on_common.append(bl)

psd_on_common      = np.array(psd_on_common)    # shape (n_lo, n_channels)
baseline_on_common = np.array(baseline_on_common)
print(f'Baseline-subtracted spectra shape: {psd_on_common.shape}')
print(f'HI mask half-width: {HI_HALF_WIDTH_HZ/1e3:.1f} kHz  (~150 km/s)')


 ** On entry to DLASCL parameter number  4 had an illegal value


/tmp/ipykernel_1759192/995243991.py:21: RuntimeWarning: invalid value encountered in divide
  f_norm   = (f - f_centre) / f_scale


LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:

# Reconstruct raw PSD = baseline-subtracted residual + baseline
raw_psd_on_common = psd_on_common + baseline_on_common   # shape (n_lo, n_channels)

# Global limits across all LOs (for shared axes)
finite_vals = raw_psd_on_common[np.isfinite(raw_psd_on_common) & (raw_psd_on_common > 0)]
y_min, y_max = finite_vals.min(), finite_vals.max()
x_min, x_max = f_common_mhz.min(), f_common_mhz.max()

x_pad = 0.05 * (x_max - x_min)
y_pad = 0.05 * (np.log10(y_max) - np.log10(y_min))

n_lo = len(lo_records)
fig, axes = plt.subplots(n_lo, 1, figsize=(11, 2.2 * n_lo),
                         sharex=True, sharey=True)

for ax, rec, raw, bl in zip(axes, lo_records, raw_psd_on_common, baseline_on_common):
    lo_mhz = rec.center_freq / 1e6

    ax.semilogy(f_common_mhz, raw, lw=0.7, color='tab:blue', alpha=0.8, label='Raw PSD')
    ax.semilogy(f_common_mhz, bl,  lw=1.4, color='tab:orange', linestyle='--', label='Baseline fit')
    ax.axvline(HI_REST_HZ / 1e6, color='gray', lw=0.8, linestyle=':', label='HI rest')

    ax.set_ylabel('PSD (counts²/bin)', fontsize=8)
    ax.set_title(f'LO = {lo_mhz:.0f} MHz', fontsize=9, loc='left')
    ax.grid(True, lw=0.4, alpha=0.5, which='both')

axes[0].set_xlim(x_min - x_pad, x_max + x_pad)
axes[0].set_ylim(10**(np.log10(y_min) - y_pad), 10**(np.log10(y_max) + y_pad))
axes[0].legend(fontsize=8, loc='upper right')
axes[-1].set_xlabel('Frequency (MHz)')

fig.suptitle('Lab 2.1 — Raw PSD with polynomial baseline fit (per LO)', y=1.01)
fig.tight_layout()
plt.show()
